In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import numpy as np 
import pandas as pd

In [3]:
dataset_path = "./dataset.csv"

df = pd.read_csv(
    dataset_path,
    # dtype is specified to prevent a DtypeWarning
    dtype={
        "full_query": str,
        "label": int,
        "statement_type": str,
        "query_template_id": str,
        "attack_payload": str,
        "attack_id": str,
        "attack_technique": str,
        "attack_desc": str,
        "split": str,
        "sqlmap_status" : str,
        "attack_stage" : str,

    },
)


In [ ]:
def display_counts_recon(df : pd.DataFrame):
    result = df.groupby('attack_id')['attack_stage'].value_counts().unstack(fill_value=0)
    display(result)
_ = display_counts_recon(df)


attack_stage,exploit,recon
attack_id,,
error-0,188,273
error-1,188,273
error-10,228,255
error-11,228,255
error-12,228,255
error-13,228,255
error-14,304,255
error-15,304,256
error-16,264,247


In [5]:
_df = df.loc[(df["attack_id"] == "error-42") & (df["attack_stage"] == "exploit")]
display(_df)

,full_query,label,attack_payload,attack_desc,attack_stage,sqlmap_status,statement_type,query_template_id,attack_id,attack_technique,split
19639,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19640,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19641,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19642,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19643,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19644,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19645,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19646,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19647,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
19648,"SELECT r.name AS region_name, COUNT(a.id) AS a...",1,NaN,NaN,exploit,success,select,airport-S16,error-42,error,train
